In [1]:
import pandas as pd

# Load train and validation datasets separately
train_df = pd.read_csv(r"C:\Users\adamf\Desktop\race classifier\fairface_label_train.csv")
val_df = pd.read_csv(r"C:\Users\adamf\Desktop\race classifier\fairface_label_val.csv")

print("Train Data Columns:", train_df.columns)  # Check if 'file' and 'race_encoded' exist
print("Validation Data Columns:", val_df.columns)



Train Data Columns: Index(['file', 'age', 'gender', 'race', 'service_test'], dtype='object')
Validation Data Columns: Index(['file', 'age', 'gender', 'race', 'service_test'], dtype='object')


In [2]:
# Convert race labels to numbers

from sklearn.preprocessing import LabelEncoder

# Encode race labels as numerical values
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
train_df['race_encoded'] = label_encoder.fit_transform(train_df['race'])
val_df['race_encoded'] = label_encoder.transform(val_df['race'])  # Use same encoding


# Mapping of race categories
# zip() pairs each category with its corresponding number and dict() creates a dictionary from the pairs fir easy reference
race_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print("Race Mapping:", race_mapping)


Race Mapping: {'Black': 0, 'East Asian': 1, 'Indian': 2, 'Latino_Hispanic': 3, 'Middle Eastern': 4, 'Southeast Asian': 5, 'White': 6}


In [5]:
import os

# Print the first few file names from the DataFrame
print("Sample filenames from train_df:")
print(train_df['file'].head())

# List some actual image files from the directory
print("\nFiles in training image directory:")
print(os.listdir(TRAIN_IMG_DIR)[:5])  # List the first 5 files


Sample filenames from train_df:
0    train/1.jpg
1    train/2.jpg
2    train/3.jpg
3    train/4.jpg
4    train/5.jpg
Name: file, dtype: object

Files in training image directory:


NameError: name 'TRAIN_IMG_DIR' is not defined

In [6]:
import cv2
import numpy as np
import os

# Define paths to image directories
TRAIN_IMG_DIR = r"C:\Users\adamf\Desktop\race classifier\fairface-img-margin025-trainval\train"
VAL_IMG_DIR = r"C:\Users\adamf\Desktop\race classifier\fairface-img-margin025-trainval\val"


# Function to load images in batches
def image_generator(df, img_dir, batch_size=64):
    while True:  # Infinite loop for batch generation
        for i in range(0, len(df), batch_size):
            batch_df = df.iloc[i:i+batch_size]  # Get batch
            images = []
            labels = []
            
            for _, row in batch_df.iterrows():
                filename = row['file'].replace("train/", "").replace("val/", "")
                img_path = os.path.join(img_dir, filename)

                img = cv2.imread(img_path)
                if img is None:
                    continue  # Skip missing images

                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
                img = img.astype('float32') / 255.0  # Normalize

                images.append(img)
                labels.append(row['race_encoded'])

            yield np.array(images), np.array(labels)  # Return batch

# Create generators for training and validation
train_gen = image_generator(train_df, TRAIN_IMG_DIR, batch_size=32)
val_gen = image_generator(val_df, VAL_IMG_DIR, batch_size=32)




In [8]:
# Get the first batch of images from the training generator
X_batch, y_batch = next(train_gen)

# Print shape of the batch to verify
print("Batch Shape:", X_batch.shape, y_batch.shape)


Batch Shape: (32, 224, 224, 3) (32,)


In [7]:
import tensorflow as tf
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available: 0


In [9]:
# CNN ARCHITECTURE

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define the CNN model
model = Sequential([
    # First Convolutional Block
    Conv2D(32, (3,3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(pool_size=(2,2)),

    # Second Convolutional Block
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),

    # Third Convolutional Block
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),

    # Fully Connected Layers
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),  # Prevent overfitting
    Dense(7, activation='softmax')  # 7 classes (one per race category)
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)


# Use below if your PC is fast
# model.fit(
#     train_gen, 
#     validation_data=val_gen, 
#     steps_per_epoch=len(train_df) // 64,  # Batches per epoch
#     validation_steps=len(val_df) // 64, 
#     epochs=10
# )

model.fit(
    train_gen,
    validation_data=val_gen,
    steps_per_epoch=500,  # Lower total steps per epoch
    validation_steps=100,  # Lower validation steps
    epochs=10
)



Epoch 1/10
500/500 [==============================] - 634s 1s/step - loss: 1.8410 - accuracy: 0.2655 - val_loss: 1.7029 - val_accuracy: 0.3634
Epoch 2/10
500/500 [==============================] - 604s 1s/step - loss: 1.6676 - accuracy: 0.3520 - val_loss: 1.5772 - val_accuracy: 0.3831
Epoch 3/10
500/500 [==============================] - 611s 1s/step - loss: 1.6175 - accuracy: 0.3681 - val_loss: 1.6118 - val_accuracy: 0.3744
Epoch 4/10
500/500 [==============================] - 610s 1s/step - loss: 1.5780 - accuracy: 0.3814 - val_loss: 1.4651 - val_accuracy: 0.4279
Epoch 5/10
500/500 [==============================] - 606s 1s/step - loss: 1.5436 - accuracy: 0.4008 - val_loss: 1.4648 - val_accuracy: 0.4263
Epoch 6/10
500/500 [==============================] - 612s 1s/step - loss: 1.5096 - accuracy: 0.4164 - val_loss: 1.4461 - val_accuracy: 0.4428
Epoch 7/10
500/500 [==============================] - 615s 1s/step - loss: 1.4835 - accuracy: 0.4219 - val_loss: 1.4182 - val_accuracy: 0.4443

In [10]:
model.save("race_classifier_model_v1.h5")


In [ ]:
from tensorflow.keras.models import load_model

# Load the saved model
model = load_model("race_classifier_model.h5")
